Load Data

In [21]:
import pandas as pd
import glob

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn import set_config
from sklearn.preprocessing import StandardScaler, MinMaxScaler

from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn

from bagpipe.preprocessing import ApplyThreshold, _ConcatDataFrames, _SeparateDataFrames, SkScalerWrapper, CreateConcatDataset

In [22]:
set_config(display="diagram")
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
device

'cuda'

In [30]:
train_dflist = []
for i, fname in enumerate(sorted(glob.glob("../data/" + "/*.csv"))):
    train_dflist.append(pd.read_csv(fname, sep=",").drop(["Date", "Dividends", "Stock Splits"], axis=1))
print(len(train_dflist))

2


In [31]:
train_dflist[0]

,Open,High,Low,Close,Volume
0,3.748967,4.283869,3.739664,4.279217,395343000
1,4.307126,4.348058,4.103398,4.179680,103044000
2,4.183400,4.184330,3.986184,4.093164,49898000
3,4.125723,4.219679,4.125723,4.180608,30002000
4,4.179678,4.474572,4.176887,4.419686,62344000
...,...,...,...,...,...
1576,40.032931,40.384470,39.846262,40.121052,4699000
1577,40.045271,40.356066,40.045271,40.232883,3983000
1578,40.268894,40.410077,39.961887,40.348488,4011000
1579,40.036730,40.171284,39.822586,39.887020,3801000


Setup Pytorch Dataset for RNN Training

In [32]:
class StockDataset(Dataset):
    def __init__(self, df, target, seq_length, dtype=torch.float32):
        self.data = df.copy(deep=True)
        self.seq_length = seq_length
        self.y = torch.tensor(self.data[target].to_numpy(), dtype=dtype)
        self.X = torch.tensor(self.data.drop([target], axis=1).to_numpy(), dtype=dtype,)

    def __getitem__(self, index):
        x = self.X[index : index + self.seq_length]
        y = self.y[index + self.seq_length - 1]  # -1 important to avoid forecasting!
        return x, y

    def __len__(self):
        return len(self.data) - self.seq_length

Preprocess Data

In [33]:
columns_to_standardize = [
    "Open",
    "High",
    "Low",
    "Close"
]

columns_to_minmax = [
    "Volume"
]

Example Pipeline 1: Only fascilitating transformers that can handle dataframe lists

In [34]:
pipeline_v1 = Pipeline([('threshold', ApplyThreshold(threshold=10, by="Volume", seq_length=10)), 
                      ('stand scaler', SkScalerWrapper(StandardScaler(), columns_to_standardize)),
                      ('minmax scaler', SkScalerWrapper(MinMaxScaler(), columns_to_minmax)),
                      ('concat dataset', CreateConcatDataset(StockDataset, target="Volume", seq_length=10)),
                     ])
pipeline_v1

Pipeline(steps=[('threshold',
                 ApplyThreshold(by='Volume', seq_length=10, threshold=10)),
                ('stand scaler',
                 SkScalerWrapper(columns=['Open', 'High', 'Low', 'Close'],
                                 scaler=StandardScaler())),
                ('minmax scaler',
                 SkScalerWrapper(columns=['Volume'], scaler=MinMaxScaler())),
                ('concat dataset',
                 CreateConcatDataset(dataset_class=<class '__main__.StockDataset'>))])

In [36]:
pipeline_v1["minmax scaler"]

SkScalerWrapper(columns=['Volume'], scaler=MinMaxScaler())

Example Pipeline 2: Merging Dataframe List via Indexing to apply global scaling with out-of-the-box sklearn transformers

In [35]:
ct = ColumnTransformer([("stand", StandardScaler(), columns_to_standardize),
                        ("minmax", MinMaxScaler(), columns_to_minmax)],
                       remainder="passthrough",
                       verbose_feature_names_out=False)

pipeline_v2 = Pipeline([('threshold', ApplyThreshold(threshold=10, by="Volume", seq_length=10)), 
                     ('concat', _ConcatDataFrames()), 
                     ('column transformer', ct.set_output(transform="pandas")),
                     ('separate', _SeparateDataFrames()),
                     ('concat dataset', CreateConcatDataset(StockDataset, target="Volume", seq_length=10)),
                     ])
pipeline_v2

Pipeline(steps=[('threshold',
                 ApplyThreshold(by='Volume', seq_length=10, threshold=10)),
                ('concat', _ConcatDataFrames()),
                ('column transformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('stand', StandardScaler(),
                                                  ['Open', 'High', 'Low',
                                                   'Close']),
                                                 ('minmax', MinMaxScaler(),
                                                  ['Volume'])],
                                   verbose_feature_names_out=False)),
                ('separate', _SeparateDataFrames()),
                ('concat dataset',
                 CreateConcatDataset(dataset_class=<class '__main__.StockDataset'>))])

Application of the Example 2 Pipeline on the train data

In [9]:
train_dataset = pipeline_v2.fit_transform(train_dflist)

Application of the fitted pipeline on the test data (to scale with mean and std of the train data (known during training))

In [10]:
# testsplit
test_dataset = pipeline_v2.transform(test_dflist)

Pass concatenated datasets to the dataloaders

In [11]:
train_dataloader = DataLoader(train_dataset, batch_size=256, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=256, shuffle=True)

Setup simple RNN Training

In [13]:
class Model(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(Model, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.rnn(x)
        out = out[:, -1, :]
        out = self.fc(out)

        return out

In [14]:
# initialize the model
input_size = 11
hidden_size = 10
num_layers = 1
output_size = 1
lr = 0.01

model = Model(input_size, 
                 hidden_size, 
                 num_layers, 
                 output_size
                 ).to(device)

# define the loss function and optimizer
criterion = nn.L1Loss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

Training Loop

In [15]:
num_epochs = 5

model.train()
for epoch in range(num_epochs):
        running_train_loss = 0.0
        for step, (x, y) in enumerate(train_dataloader):
                X, y = x.to(device), y.to(device).reshape([len(y),1])

                optimizer.zero_grad()
                pred = model(X)
                
                train_loss = criterion(pred, y)
                train_loss.backward()
                
                optimizer.step()
        
                print(f'EPOCH {epoch+1} of {num_epochs} batch {step+1} of {len(train_dataloader)} loss: {round(train_loss.item(),4)} ', end="\r")
        print("")

EPOCH 1 of 5 batch 159 of 159 loss: 0.0523 
EPOCH 2 of 5 batch 159 of 159 loss: 0.0423 
EPOCH 3 of 5 batch 159 of 159 loss: 0.0417 
EPOCH 4 of 5 batch 159 of 159 loss: 0.0411 
EPOCH 5 of 5 batch 159 of 159 loss: 0.042  


Testing

In [16]:
model.eval()
correct = 0
total = 0
total_loss = 0  
with torch.no_grad():
    for step, (x, y) in enumerate(test_dataloader):
        X, y = x.to(device), y.to(device).reshape([len(y),1])

        pred = model(X)
        loss = criterion(pred, y)
        total_loss += loss.item()

    print(f"Avg. Loss: {total_loss/step}")

Avg. Loss: 0.07833668961517234
